## Testing Julia for GAIA

using the julia language for GAIA analaysis

In [4]:
using PyCall
using DataFrames

@pyimport astropy.io.votable as votable
@pyimport matplotlib.pyplot as plt

## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "/home/stephane/Science/cluster/GAIA/products"

cd(wdir)

In [5]:
## function to create the df and filter the data

function read_votable(voname::String)
    
    vot = votable.parse(voname)
    data = vot[:get_first_table]() 
    
    return(data["array"]["data"])

function filter_data(gaia_data, dist_range = [0., 2000], vra_range = [-200,200], vdec_range = [-200.,200], mag_range =[-1e9, 1e9])

    1
    
end


LoadError: syntax: incomplete: "function" at In[5]:1 requires end

In [6]:
voname = "NGC 2516-1.0deg.vot"
vot = votable.parse(voname)

T = typeof(vot)
for (name, typ) in zip(fieldnames(T), T.types)
     println("type of the fieldname $name is $typ")
 end
 
b= vot[:get_first_table]() 

# c = convert(Array, b::PyObject)
#print(b)
print(keys(b::PyObject))


# print(b["array"]["data"])

#for s in b["array"]["data"]
#    println(s)
# end


mutable struct df
    ndata::Int32
    data::Array{Float64, Float64}
end

x = range(0; stop=2*pi, length=1000); y = sin.(3 * x + 4 * cos.(2 * x));
plt.plot(x, y, color="red", linewidth=2.0, linestyle="--")
plt.show()

begin
    local ind ::Int32 = 1
    println(length(b["array"]["data"]))
    
    lgal = convert(Float64,b["array"]["data"][1]["l"])
    bgal = convert(Float64,b["array"]["data"][1]["b"])
    parallax = convert(Float64,b["array"]["data"][1]["parallax"]) 
    parallax_err = convert(Float64,b["array"]["data"][1]["parallax"]) 
    pmra  = convert(Float64,b["array"]["data"][1]["pmra"])
    pmra_err  = convert(Float64,b["array"]["data"][1]["pmra_error"])
    pmdecc  = convert(Float64,b["array"]["data"][1]["pmdec"])
    pmdec_err  = convert(Float64,b["array"]["data"][1]["pmdec_error"])
    
    G = convert(Float64,b["array"]["data"][ind]["phot_g_mean_mag"])
    Rp = convert(Float64,b["array"]["data"][ind]["phot_rp_mean_mag"])
    Bp = convert(Float64,b["array"]["data"][ind]["phot_bp_mean_mag"])
    
    println(G)
    
end


type of the fieldname o is Ptr{PyCall.PyObject_struct}
Symbol[:ID, :_ID, :__bytes__, :__class__, :__delattr__, :__dict__, :__dir__, :__doc__, :__eq__, :__format__, :__ge__, :__getattribute__, :__gt__, :__hash__, :__init__, :__init_subclass__, :__le__, :__lt__, :__module__, :__ne__, :__new__, :__reduce__, :__reduce_ex__, :__repr__, :__setattr__, :__sizeof__, :__str__, :__subclasshook__, :__weakref__, :_add_definitions, :_add_field, :_add_group, :_add_info, :_add_link, :_add_param, :_add_unknown_tag, :_attr_list, :_config, :_description, :_element_name, :_empty, :_fields, :_format, :_get_binary_data_stream, :_groups, :_ignore_add, :_infos, :_links, :_name, :_nrows, :_params, :_parse_binary, :_parse_fits, :_parse_tabledata, :_pos, :_ref, :_resize_strategy, :_ucd, :_ucd_in_v1_2, :_votable, :_write_binary, :_write_tabledata, :array, :create_arrays, :description, :fields, :format, :from_table, :get_field_by_id, :get_field_by_id_or_name, :get_fields_by_utype, :get_group_by_id, :get_groups_by_

KeyError: KeyError: key "phot_g_mean_mag_error" not found